<a href="https://colab.research.google.com/github/can67yang/sakura-own/blob/main/Sakura-ASMR-cloab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 初始化环境
#@markdown 挂载Google网盘
Mount_GDrive = True # @param {type:"boolean"}
if Mount_GDrive:
  from google.colab import drive

  drive.mount('/content/gdrive')
  ROOT_PATH = "/content/gdrive/MyDrive"
else:
  ROOT_PATH = "/content"
!nvidia-smi

In [ ]:
#@title 安装依赖
%cd $ROOT_PATH
!git clone https://github.com/SakuraLLM/SakuraLLM.git

%cd SakuraLLM
!git pull

LLAMA_CPP = True # @param {type:"boolean"}
VLLM = False # @param {type:"boolean"}
if LLAMA_CPP:
  !pip install "diskcache>=5.6.1"
  !pip install llama-cpp-python -i https://abetlen.github.io/llama-cpp-python/whl/cu122
if VLLM:
  !pip install -U transformers tokenizers
  !pip install vllm
!pip install -q -r requirements.txt
!pip install -q pyngrok

In [ ]:
#@title 调用api
from huggingface_hub import hf_hub_download
from pyngrok import ngrok

# 设置 ngrok authtoken（替换 <YOUR_AUTH_TOKEN> 为你的实际 ngrok authtoken）
!ngrok authtoken 2aD0E5ADxy6miRBapzrwWTOu3j6_3Phqsv4LxCQwu7iVuxYgT

# 启动 ngrok 隧道并连接到本地端口 8181
public_url = ngrok.connect(8181)
print("ngrok tunnel URL:", public_url)

repo_id = "SakuraLLM/Sakura-14B-Qwen2.5-v1.0-GGUF"
MODEL = "sakura-14b-qwen2.5-v1.0-q4km.gguf" # @param ["sakura-14b-qwen2.5-v1.0-iq4xs.gguf", "sakura-14b-qwen2.5-v1.0-q4km.gguf", "sakura-14b-qwen2.5-v1.0-q6k.gguf"]
hf_hub_download(repo_id=repo_id, filename=MODEL, local_dir="models/")
MODEL_PATH = f"./models/{MODEL}"

%cd $ROOT_PATH/SakuraLLM
!python server.py \
    --model_name_or_path $MODEL_PATH \
    --llama_cpp \
    --use_gpu \
    --model_version 1.0 \
    --trust_remote_code \
    --no-auth \
    --listen localhost:8181

In [ ]:
#@title 翻译文本文件
from huggingface_hub import hf_hub_download

repo_id = "SakuraLLM/Sakura-14B-Qwen2beta-v0.9.2-GGUF"
MODEL = "sakura-14b-qwen2beta-v0.9.2-q4km.gguf" # @param ["sakura-14b-qwen2beta-v0.9.2-iq4xs.gguf", "sakura-14b-qwen2beta-v0.9.2-q2k.gguf", "sakura-14b-qwen2beta-v0.9.2-q3km.gguf", "sakura-14b-qwen2beta-v0.9.2-q4km.gguf", "sakura-14b-qwen2beta-v0.9.2-q6k.gguf"]
hf_hub_download(repo_id=repo_id, filename=MODEL, local_dir="models/")
MODEL_PATH = f"./models/{MODEL}"
DATA_PATH = "novel.txt" # @param {type:"string"}
OUTPUT_PATH = "novel_translated.txt" # @param {type:"string"}

%cd $ROOT_PATH/Sakura-13B-Galgame
!python translate_novel.py \
    --model_name_or_path $MODEL_PATH \
    --llama_cpp \
    --use_gpu \
    --model_version 0.9 \
    --trust_remote_code \
    --data_path $DATA_PATH \
    --output_path $OUTPUT_PATH